In [7]:
import numpy as np
from scipy.stats import truncnorm
import src.models.helpers as mh


In [19]:
def get_noise_model_CPT(pVar: mh.VariableNode, cVar: mh.VariableNode, c_resolution: int, std_gauss: float, n: int, debug=False):
    """
    pVar: parent variable, corresponds to the true value that would be obtained for that variable if we had a magic measurement devive
    cVar: child variable, corresponds to the measured value, or a value that could be obtained with a measurement device from clinical practice
    c_resolution: number of decimals, correspond to the resolution of the measurement device
    std_gauss: standard deviation of the gaussian noise of the measurement device
    n: number of samples to generate per bin of the parent variable
    """
    cpt = np.zeros((cVar.card, pVar.card))
    for i, pVar_bin in enumerate(pVar.get_bins_arr()):
        real_arr = pVar.sample_from_bin(pVar_bin, n)
        # Add gaussian noise
        # Truncnorm ensures that sampled values are plausible (within the possible values of the child variable)
        noisy_real_arr = [truncnorm(cVar.a, cVar.b, loc, scale=1).rvs(1) for loc in real_arr]
        # Round to measurement resolution
        rounded_arr = np.round(noisy_real_arr, c_resolution)
        # Discard generated values that are outside the measurement device boundaries
        rounded_arr = rounded_arr[(rounded_arr >= cVar.a) & (rounded_arr <= cVar.b)]
        # bin up the rounded values to the child variable
        p_cVar_given_pVar_obs = cVar.bin_up(rounded_arr, normalise=True)
        print(f"p_cVar_given_pVar_obs: {p_cVar_given_pVar_obs.shape}")
        # Add to the CPT
        cpt[:, i] = p_cVar_given_pVar_obs

        assert np.isclose(
            cpt[:, i].sum(), 1, atol=cVar.tol
        ), f"The sum of probabilities should be 1, got {cpt[:, i].sum()} while calculating P({cVar.name}|{pVar.name}={pVar_bin})"
    return cpt


In [20]:
ecFEV1 = mh.VariableNode("ecFEV1 (L)", 0, 6, 0.05, prior={"type": "uniform"})
uecFEV1 = mh.VariableNode("Underlying ecFEV1 (L)", 0, 6, 0.05, prior=None)
std_gauss = 0.3
n_decimals = 2

get_noise_model_CPT(uecFEV1, ecFEV1, n_decimals, std_gauss, 1000, debug=True)

p_cVar_given_pVar_obs: (119,)


ValueError: could not broadcast input array from shape (119,) into shape (120,)

In [24]:
ecFEV1.bins

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05,
       1.1 , 1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 ,
       1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05, 2.1 , 2.15,
       2.2 , 2.25, 2.3 , 2.35, 2.4 , 2.45, 2.5 , 2.55, 2.6 , 2.65, 2.7 ,
       2.75, 2.8 , 2.85, 2.9 , 2.95, 3.  , 3.05, 3.1 , 3.15, 3.2 , 3.25,
       3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 , 3.75, 3.8 ,
       3.85, 3.9 , 3.95, 4.  , 4.05, 4.1 , 4.15, 4.2 , 4.25, 4.3 , 4.35,
       4.4 , 4.45, 4.5 , 4.55, 4.6 , 4.65, 4.7 , 4.75, 4.8 , 4.85, 4.9 ,
       4.95, 5.  , 5.05, 5.1 , 5.15, 5.2 , 5.25, 5.3 , 5.35, 5.4 , 5.45,
       5.5 , 5.55, 5.6 , 5.65, 5.7 , 5.75, 5.8 , 5.85, 5.9 , 5.95])

In [14]:
truncnorm(0, 6, loc=0, scale=1).rvs(100)

array([0.5863966 , 0.79719265, 0.45352517, 0.44620108, 1.4103153 ,
       0.98355876, 1.99514937, 0.08067985, 0.04122751, 1.73382982,
       1.48368782, 0.02052026, 1.25439054, 1.04572573, 1.46228047,
       2.97239652, 1.33975757, 0.55570181, 1.49220099, 0.62864993,
       0.42100405, 0.32379648, 0.49904411, 3.44427336, 0.65011594,
       0.43890812, 1.17707563, 0.34548403, 0.88327586, 0.99093691,
       0.46816499, 1.13532727, 1.63202868, 2.14361015, 0.9835354 ,
       1.18543274, 0.28065774, 0.28366307, 1.88067664, 0.39993263,
       1.04052175, 0.60987415, 0.91105782, 0.8575957 , 1.67733063,
       0.56134619, 0.53759807, 0.15347993, 1.76850208, 1.2276485 ,
       0.04722612, 1.59827263, 1.10075014, 0.41855256, 1.00980843,
       0.3966443 , 1.76349627, 0.1532164 , 3.29427437, 1.9626444 ,
       1.31981906, 0.23877987, 1.20797685, 0.12012817, 0.34875067,
       1.00669862, 0.18260326, 1.22800898, 1.37299755, 1.29081516,
       3.41007208, 0.7055521 , 0.03436142, 2.48382423, 1.79251